In [13]:
import pandas as np
import numpy as np1 
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn import svm, datasets, metrics

In [14]:
def normalize(data, cat) :
    data[cat]= ((data[cat]-data[cat].min())/ (data[cat].max()-data[cat].min()))

In [15]:
df= np.read_csv('education.csv')
df_1= np.read_csv('educnorma.csv',index_col=0)

In [16]:
y = df["Performance in online"].copy()
df_1.drop(["performance_online"],axis=1,inplace=True)

In [17]:
y

0        6
1        3
2        6
3        4
4        6
        ..
1028     8
1029    10
1030     6
1031     6
1032    10
Name: Performance in online, Length: 1033, dtype: int64

In [18]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033 entries, 0 to 1032
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           1033 non-null   float64
 1   nb_subjets                    1033 non-null   float64
 2   Family_size                   1033 non-null   float64
 3   Internet                      1033 non-null   float64
 4   Study                         1033 non-null   float64
 5   Sleep                         1033 non-null   float64
 6   Media                         1033 non-null   float64
 7   Note_avant                    1033 non-null   float64
 8   Online_mode                   1033 non-null   float64
 9   doubts_online                 1033 non-null   float64
 10  Gender_Male                   1033 non-null   int64  
 11  Home_location_Urban           1033 non-null   int64  
 12  Education_Post Graduate       1033 non-null   int64  
 13  Edu

In [19]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [29]:
import math
from random import seed, shuffle
SEED = 1122334455
seed(SEED)
np1.random.seed(SEED)


def reapeated_random(n_iter,df_1) :
    X = df_1.to_numpy(copy=True)
    y1 = y.to_numpy(copy=True)   
    matrise_result=[] # dim 1 -> interation 
                     # dim 2 -> 0: training ; 1:testing ; 2:validation
                     # dim 3 -> c1,c2,c3,c4,c5
    C_i=[0.001,1,100,1000,1]
    C_range=np1.shape(C_i)[0]
    for i in range (n_iter) :
        print(i,end='')
        n_size=np1.shape(X)[0]
        perm = list(range(0,n_size))
        shuffle(perm)
        X_syn1 = X[perm]
        y_syn1 = y[perm]
        #Generate training, testing and validation:
        n_train=int(0.5*n_size)
        n = np1.shape(X_syn1)[0]
        X_syn_training=X_syn1[0:n_train,:]
        y_syn_training=y_syn1[0:n_train]

        X_syn_testing=X_syn1[n_train:(n_train+math.floor((n-n_train)/2)),:]
        y_syn_testing=y_syn1[n_train:(n_train+math.floor((n-n_train)/2))]

        X_syn_validation=X_syn1[(n_train+math.floor((n-n_train)/2)):n,:]
        y_syn_validation=y_syn1[(n_train+math.floor((n-n_train)/2)):n]

        
        #resolution
        Accuracy_training=np1.empty([C_range])
        Accuracy_testing=np1.empty([C_range])
        Accuracy_validation=np1.empty([C_range])
        
        y_syn2_testing = y_syn_testing.copy()
        for i in y_syn2_testing.keys() :
            if y_syn2_testing[i] < 7: y_syn2_testing[i]= 0
            else : y_syn2_testing[i]=1
                
        y_syn2_validation = y_syn_validation.copy()
        for i in y_syn2_validation.keys() :
            if y_syn2_validation[i] < 7: y_syn2_validation[i]= 0
            else : y_syn2_validation[i]=1
                
        for i in range(C_range):
    
            svm_clf = SVC(kernel="linear", C=C_i[i])
            svm_clf.fit(X_syn_training, y_syn_training)
  
            y_pred_training=svm_clf.predict(X_syn_training)
            y_pred_testing=svm_clf.predict(X_syn_testing)
            y_pred_validation=svm_clf.predict(X_syn_validation)
            
            for j in range (len(y_pred_validation)):
                if y_pred_validation[j] < 7: y_pred_validation[j]= 0
                else : y_pred_validation[j]=1
            for j in range (len(y_pred_testing)):
                if y_pred_testing[j] < 7: y_pred_testing[j]= 0
                else : y_pred_testing[j]=1       
            Accuracy_training[i]=(metrics.accuracy_score(y_syn_training, y_pred_training)*100)
            Accuracy_testing[i]=(metrics.accuracy_score(y_syn2_testing, y_pred_testing)*100)
            Accuracy_validation[i]=(metrics.accuracy_score(y_syn2_validation, y_pred_validation)*100)   
        
        matrise_result += [[Accuracy_training,Accuracy_testing,Accuracy_validation]]
    return matrise_result


In [30]:
def final_accuracy(matrice):
    Final_accuracy=[]
    for i in range (10):
        max_t=max(matrice[i][1])
        i_loc=np1.where(matrice[i][1]>=max_t)
        if len(i_loc[0])== 1 : 
            i_loc=i_loc[0][0]
        else : 
            acc = []
            for x in i_loc[0] : 
                acc += [matrice[i][2][x]]
            #print(acc)
            max_t = max(acc)
            i_acc = np1.where(acc>=max_t)
            i_loc= i_loc[0][i_acc[0][0]]
        
        Final_accuracy += [matrice[i][2][i_loc]]
    #print(Final_accuracy)
    print("La précision moyenne est  %.2f" %np1.mean(Final_accuracy))
    print("La variance de précision est %.2f" %np1.var(Final_accuracy))

In [31]:
df_2 = df_1[["Study","Sleep","Media","Note_avant"]].copy()
matrice = np1.array(reapeated_random(10,df_2))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  48.03
La variance de précision est 15.69


In [32]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  65.56
La variance de précision est 1.66


In [33]:
df_1.drop([ "nb_subjets" , "Sleep", "Note_avant","Home_location_Urban","Education_Post Graduate","Education_School" \
           ,"Education_Under Graduate","Economic_status_Middle Class","Economic_status_Poor","Economic_status_Rich", \
           "Sport_Yes", "Gaming_Yes","interest_Both"],axis=1,inplace=True)

In [34]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  67.07
La variance de précision est 6.59


In [35]:
df_1.drop(["group_study_yes","separate_room_Yes","Monitoring_Yes","Gender_Male", "Family_size", "Media","Device_used_Laptop" \
           ,"Device_used_Mobile","interest_Practical","interest_Theory","Device_used_Desktop"],axis=1,inplace=True)

In [36]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  66.91
La variance de précision est 10.94


In [37]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033 entries, 0 to 1032
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          1033 non-null   float64
 1   Internet                     1033 non-null   float64
 2   Study                        1033 non-null   float64
 3   Online_mode                  1033 non-null   float64
 4   doubts_online                1033 non-null   float64
 5   Satisfaction_online_Average  1033 non-null   int64  
 6   Satisfaction_online_Bad      1033 non-null   int64  
 7   Satisfaction_online_Good     1033 non-null   int64  
dtypes: float64(5), int64(3)
memory usage: 72.6 KB


In [38]:
df_1.drop(["Age","Study","Satisfaction_online_Average"],axis=1,inplace=True)

In [39]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033 entries, 0 to 1032
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Internet                  1033 non-null   float64
 1   Online_mode               1033 non-null   float64
 2   doubts_online             1033 non-null   float64
 3   Satisfaction_online_Bad   1033 non-null   int64  
 4   Satisfaction_online_Good  1033 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 48.4 KB


In [40]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  68.53
La variance de précision est 15.60


In [41]:
df_1.drop(["Online_mode","Satisfaction_online_Bad","Satisfaction_online_Good"],axis=1,inplace=True)

In [42]:
matrice = np1.array(reapeated_random(10,df_1))
print('')
final_accuracy(matrice)

0123456789
La précision moyenne est  67.76
La variance de précision est 13.99
